## Applying Machine Learning to Trading Strategies: Using Logistic Regression to Build Momentum-based Trading Strategies - <br>`Patrick Beaudan and Shuoyuan He`

Objectives :

    1. Use of ML Model, Logistic Regression, to build a time-series dual momentum trading strategy on the S&P 500 Index
    2. Showing how the proposed model outperforms both buy-and-hold and several base-case dual momentum strategies, significantly increasing returns and reducing risk
    3. Applying the algorithm to other U.S. and international large capitalization equity indices 
    4. Analyzing yields improvements in risk-adjusted performance. 

### 1. Fetching data

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline 
plt.style.use('seaborn-v0_8-dark-palette')
import yfinance as yf 
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, log_loss

import warnings
warnings.filterwarnings('ignore') 

#### Tickers 
1. S&P 500 Index: `^GSPC`
2. S&P Small Cap 600 Index (SML): ^SML   
3. S&P Mid Cap 400 Index (MID): `^MID`
4. FTSE 100 Index (UKX): `^FTSE`
5. FTSEurofirst 300 Index (E300): ^FTEU3  
6. Tokyo Stock Exchange Price Index (TPX): ^TPX  
7. Dow Jones Industrial Average Index (INDU): `^DJI`
8. Dow Jones Transportation Average Index (TRAN): `^DJT`

In [2]:
end = '2018-12-12'

# df_sml = yf.download('^SML',start='1993-12-31',end=end) ==> Data not available
# df_mid = yf.download('^MID',start='1990-12-31',end=end) 
# df_mid.to_csv('SPMidCap400.csv')
# df_ukx = yf.download('^FTSE',start='1997-12-19',end=end)
# df_ukx.to_csv('FTSE100.csv') 
# df_e300 = yf.download('^FTEU3',start='1985-12-31',end=end) ==> Data not available
# df_tpx = yf.download('^TPX',start='1997-12-19',end=end) ==> Data not available
# df_dji = yf.download('^DJI',start='1920-01-02',end=end)
# df_dji.to_csv('DJIndustry.csv')
# df_djt = yf.download('^DJT',start='1920-01-02',end=end) 
# df_djt.to_csv('DJTransport.csv')
# df_sp500 = yf.download('^GSPC',start='1927-12-30',end=end)
# df_sp500.to_csv('SP500.csv')  

In [3]:
df_mid = pd.read_csv('SPMidCap400.csv')
df_mid.set_index('Date', inplace=True)

df_ukx = pd.read_csv('FTSE100.csv')
df_ukx.set_index('Date', inplace=True) 

df_dji = pd.read_csv('DJIndustry.csv')
df_dji.set_index('Date', inplace=True)

df_djt = pd.read_csv('DJTransport.csv') 
df_djt.set_index('Date', inplace=True) 

data = pd.read_csv('SP500.csv')
data.set_index('Date',inplace=True) 

df_21 = data.copy() 

print('Shape of data : ',data.shape) 
data.tail(3) 

Shape of data :  (22844, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-07,2691.260010,2708.540039,2623.139893,2633.080078,2633.080078,4242240000
2018-12-10,2630.860107,2647.510010,2583.229980,2637.719971,2637.719971,4162880000
2018-12-11,2664.439941,2674.350098,2621.300049,2636.780029,2636.780029,3963440000


### 2. Defining class to include base-features Momentum and Drawdown

* Momentum features are calculated over time frames of 30, 60, 90, 120, 180, 270, 300, 360 days
* Drawdown features are calculated over time frames of 15, 60, 90, 120 days

Also, it is instructed to calculate features by skipping last month. We follow the convention of 252 business days per calendar year and 21 business days per calendar month.

Features are selected based on the fact that observing the change in the shape of the price history using multiple historical time windows for momenta and drawdowns is more pertinent than considering other metrics to predict short-term profitability. So, we use momenta and drawdowns of different timeframes as features

In [4]:
class IncludeFeatures:
    def __init__(self,data):
        self.data = data 

    def calculate_momentum(self,window): # computing the rate of change in the stock's closing price over window days
        self.data[f'momntm_{window}'] =  self.data['Adj Close'] - self.data['Adj Close'].shift(window) 

    def calculate_drawdown(self,window): # Compute the drawdown by finding the peak and trough in the price data
        # calculating cumulative maximum for stocks price
        self.data['Cumulative_Peak'] = self.data['Adj Close'].cummax() # max of cumulative value upto that day
        # calculating drawdown 
        self.data[f'drwdwn_{window}'] = (self.data['Adj Close']-self.data['Cumulative_Peak'])/self.data['Cumulative_Peak']

    def include_features(self):
        
        momentum_windows = [30, 60, 90, 120, 180, 270, 300, 360]
        drawdwn_windows = [15, 60, 90, 120]    

        for days in momentum_windows:
            self.calculate_momentum(days) 

        for days in drawdwn_windows:
            self.calculate_drawdown(days) 
        
        self.data.drop(columns=['Cumulative_Peak','Open','High','Low','Close','Volume'],axis=1,inplace=True)
        return self.data     

In [5]:
include_feat = IncludeFeatures(data) 
data_feat = include_feat.include_features()
data_feat.dropna(inplace=True)
print(data_feat.shape) 
data_feat.head(3) 

(22484, 13)


,Adj Close,momntm_30,momntm_60,momntm_90,momntm_120,momntm_180,momntm_270,momntm_300,momntm_360,drwdwn_15,drwdwn_60,drwdwn_90,drwdwn_120
Date,,,,,,,,,,,,,
1929-06-10,25.270000,-0.309999,-0.459999,-0.090000,2.74,4.090000,5.060001,6.380001,7.610001,-0.041714,-0.041714,-0.041714,-0.041714
1929-06-11,25.430000,-0.100000,-0.650000,-0.020000,2.99,4.250000,5.070000,6.480000,7.670000,-0.035647,-0.035647,-0.035647,-0.035647
1929-06-12,25.450001,-0.490000,-0.590000,-0.289999,2.75,4.230001,5.010000,6.170000,7.730001,-0.034888,-0.034888,-0.034888,-0.034888


In [6]:
print(f'Null values : {data_feat.isna().sum().sum()}') 

Null values : 0


### 3. Analyzing Key Performance Indicators over sample indices over the entire period

KPIs analysed here are Annual Return, Sharpe Ratio, Volatility, Maximum Drawdown, Average Daily Drawdown

In [7]:
class KPIs:
    def __init__(self,data):
        self.datac = data  

    def annual_return(self,datac):
        cumulative_returns = (1+datac['Daily_Return']).prod()-1 
        n_days = datac.shape[0]     # Number of trading days 
        annualized_return = (1+cumulative_returns)**(252/n_days)-1
        return annualized_return 
    
    def sharpe_ratio(self,datac):
        average_return = datac['Daily_Return'].mean() 
        risk_free_rate = 0.01/252  # constant 1% annual risk-free rate
        std_dev = datac['Daily_Return'].std() 
        sharpe_ratio = (average_return-risk_free_rate)*10/std_dev
        return sharpe_ratio 

    def volatility(self,datac):
        daily_volatility = datac['Daily_Return'].std()
        trading_days_per_year = 252 
        annual_volatility = daily_volatility*np.sqrt(trading_days_per_year)   # Annualizing Volatility
        return annual_volatility 
    
    def max_drawdown(self,datac):
        datac['Running_max'] = datac['Adj Close'].cummax() 
        datac['Drawdowns'] = (datac['Adj Close']-datac['Running_max'])/datac['Running_max']

        max_drawdown = datac['Drawdowns'].min() 
        avg_drawdown = datac['Drawdowns'].mean() 

        return max_drawdown, avg_drawdown 

    def calculate_kpi(self):        
        self.datac['Log_Return'] =  np.log(self.datac['Adj Close']/self.datac['Adj Close'].shift(1))
        self.datac['Daily_Return'] = self.datac['Adj Close'].pct_change() 
        self.datac.dropna(inplace=True) 

        annualized_return = self.annual_return(self.datac)
        sharpe_ratio = self.sharpe_ratio(self.datac)
        annual_volatility = self.volatility(self.datac)
        max_drawdown, avg_drawdown = self.max_drawdown(self.datac)

        print(f'Annual Return : {annualized_return*100:.1f}%')
        print(f'Sharpe Ratio : {sharpe_ratio:.2f}')
        print(f'Volatility : {annual_volatility*100:.0f}%')
        print(f'Maximum Drawdown : {max_drawdown*100:.0f}%')
        print(f'Average Daily Drawdown : {avg_drawdown*100:.0f}%') 

        self.datac.drop(columns=['Log_Return','Daily_Return','Running_max','Drawdowns'],axis=1,inplace=True)  

In [8]:
calc_indices = {'S&P 500':data, 'S&P Mid Cap':df_mid, 'FTSE 100':df_ukx, 'DJ Industry':df_dji, 
                'DJ Transport':df_djt}

for ind,df in calc_indices.items():
    print() 
    print('='*20,f'Performace Metrics of {ind}','='*20)
    print() 
    calc_kpi = KPIs(df)
    calc_kpi.calculate_kpi()        


==================== Performace Metrics of S&P 500 ====================

Annual Return : 5.3%
Sharpe Ratio : 0.20
Volatility : 19%
Maximum Drawdown : -86%
Average Daily Drawdown : -22%

==================== Performace Metrics of S&P Mid Cap ====================

Annual Return : 10.8%
Sharpe Ratio : 0.37
Volatility : 19%
Maximum Drawdown : -56%
Average Daily Drawdown : -7%

==================== Performace Metrics of FTSE 100 ====================

Annual Return : 1.5%
Sharpe Ratio : 0.07
Volatility : 19%
Maximum Drawdown : -53%
Average Daily Drawdown : -16%

==================== Performace Metrics of DJ Industry ====================

Annual Return : 7.9%
Sharpe Ratio : 0.30
Volatility : 17%
Maximum Drawdown : -54%
Average Daily Drawdown : -9%

==================== Performace Metrics of DJ Transport ====================

Annual Return : 7.7%
Sharpe Ratio : 0.25
Volatility : 23%
Maximum Drawdown : -61%
Average Daily Drawdown : -13%


# I. Buy and Hold Trading Strategy

Trading every day for last one year. Compute returns 

In [9]:
databnh = data.copy()
databnh['Momentum'] = (databnh['Adj Close'].pct_change(periods=252)*100).round(2)  # 252 days 
databnh.dropna(inplace=True)
databnh = databnh.iloc[-252:]  # Selecting Values for last one year 

In [10]:
databnh = databnh[['Adj Close','Momentum']]
databnh['Signal'] = 0

for i in range(len(databnh)):
    if databnh['Momentum'].iloc[i] >= 5:
        databnh.loc[databnh.index[i], 'Signal'] = 1
    else:
        databnh.loc[databnh.index[i], 'Signal'] = -1    

In [11]:
print('Shape before : ', databnh.shape)
databnh_1 = databnh[databnh['Signal']==1]  
print('Shape after : ', databnh_1.shape) 

Shape before :  (252, 3)
Shape after :  (233, 3)


In [12]:
databnh_1.head() 

,Adj Close,Momentum,Signal
Date,,,
2017-12-11,2659.989990,17.72,1
2017-12-12,2664.110107,18.04,1
2017-12-13,2662.850098,17.22,1
2017-12-14,2652.010010,17.70,1
2017-12-15,2675.810059,18.29,1


In [13]:
print('='*20,'Statistics of SPX for one year','='*20) 
print() 
calc_kpi = KPIs(databnh)
calc_kpi.calculate_kpi()  
print('='*20,'Statistics for Buy and Hold for one year revisiting signal every day','='*20) 
print() 
calc_kpi = KPIs(databnh_1)
calc_kpi.calculate_kpi()  

==================== Statistics of SPX for one year ====================

Annual Return : -0.9%
Sharpe Ratio : -0.03
Volatility : 16%
Maximum Drawdown : -10%
Average Daily Drawdown : -4%
==================== Statistics for Buy and Hold for one year revisiting signal every day ====================

Annual Return : 5.3%
Sharpe Ratio : 0.23
Volatility : 14%
Maximum Drawdown : -10%
Average Daily Drawdown : -4%


# II. Classical Time Series Dual-Momentum Trading Strategy

#### Strategy

1. The momentum, i.e. the percentage price change of a security, is calculated over a historical time horizon of twelve months, skipping the most recent month 
2. If momentum > threshold (here,5%=0.05) => Invest 
3. If momentum < threshold => the portfolio is moved to cash in the long-only strategy, or moved to a short position in the long-short strategy 
4. This investment decision is revisited at regular intervals of one month 

Calculating momentum, percentage change

In [14]:
trading_days_per_month = 21
no_of_months = 12  
time_horizon = trading_days_per_month*no_of_months # 252 days 

df_21['Momentum'] = df_21['Adj Close'].pct_change(periods=252)*100
df_21.dropna(inplace=True)
df_21.drop(columns=['Open','High','Low','Close','Volume', 'Adj Close'],axis=1,inplace=True)

df_21.head(3) 

,Momentum
Date,
1929-01-03,40.770107
1929-01-04,39.921172
1929-01-07,36.851021


### Signals are generated every 21 days

In [15]:
def generate_signals(df_g, interval, threshold = 5):
    df_g['Signal'] = 0

    for i in range(interval, len(df_g), interval):
        if df_g['Momentum'].iloc[i] >= threshold:
            df_g['Signal'].iloc[i] = 1  # Invest
        else:
            df_g['Signal'].iloc[i] = -1 # Move to cash or short position

    return df_g 

df_21 = generate_signals(df_21, 1, 5) 
df_21.head() 

,Momentum,Signal
Date,,
1929-01-03,40.770107,0
1929-01-04,39.921172,1
1929-01-07,36.851021,1
1929-01-08,37.720804,1
1929-01-09,38.958104,1


In [16]:
n = len(df_21)
# Slice the DataFrame to exclude the last 21 rows for skipping most recent month 
df_x = df_21.copy() 
df_21 = df_x.iloc[:n-252] 
df_1yr = df_x.iloc[-252:]  

In [17]:
df_21.shape

(22340, 2)

In [18]:
df_1yr.shape

(252, 2)

In [19]:
df_21 = df_21[df_21['Signal']!=0] 
df_1yr = df_1yr[df_1yr['Signal']!=0]

In [20]:
print(df_21['Signal'].value_counts())  

Signal
 1    13176
-1     9163
Name: count, dtype: int64


In [21]:
print(df_1yr['Signal'].value_counts())  

Signal
 1    233
-1     19
Name: count, dtype: int64


## Machine Learning Approach

### 4. Defining Function to create polynomial features

In [22]:
def degree(data,degree): 

    feature_names = data.columns 
    # feature_names = ['Adj Close', 'momntm_30', 'momntm_60', 'momntm_90', 'momntm_120',
    #                  'momntm_180', 'momntm_270', 'momntm_300', 'momntm_360', 'drwdwn_15',
    #                  'drwdwn_60', 'drwdwn_90', 'drwdwn_120'] 
    
    if data.shape[1] != len(feature_names):
        raise ValueError("The number of features in the data does not match the length of feature names.")

    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_feat = poly.fit_transform(data) 
    
    feature_names_poly = poly.get_feature_names_out(input_features=feature_names)
    
    df_poly = pd.DataFrame(poly_feat, columns=feature_names_poly, index=data.index) 
    print(f'Shape of df_poly of degree 1 : ',data.shape) 
    print(f'Shape of df_poly of degree {degree} : ',df_poly.shape) 
    print('Number of duplicate columns : ',len(df_poly.columns)-len(set(df_poly.columns))) 
    return df_poly 

In [23]:
x_quad = degree(data_feat,2)  

Shape of df_poly of degree 1 :  (22483, 13)
Shape of df_poly of degree 2 :  (22483, 104)
Number of duplicate columns :  0


In [24]:
x_cubic = degree(data_feat,3) 

Shape of df_poly of degree 1 :  (22483, 13)
Shape of df_poly of degree 3 :  (22483, 559)
Number of duplicate columns :  0


### 5. Creating Datasets for training with Target Variable

#### 5.1 Linear dataset

In [25]:
print('Shape of linear dataset before concatenation : ',data_feat.shape)
x_linear = df_21.join(data_feat) 
x_linear.dropna(inplace=True)  
print('Shape of linear dataset after concatenation : ',x_linear.shape) 

Shape of linear dataset before concatenation :  (22483, 13)
Shape of linear dataset after concatenation :  (22231, 15)


In [26]:
print('Shape of linear dataset before concatenation : ',data_feat.shape)
xp_linear = df_1yr.join(data_feat) 
xp_linear.dropna(inplace=True)  
print('Shape of linear dataset after concatenation : ',xp_linear.shape) 

Shape of linear dataset before concatenation :  (22483, 13)
Shape of linear dataset after concatenation :  (252, 15)


#### 5.2 Quadratic dataset

In [27]:
xp_quad =  x_quad.copy() 
print('Shape of quadratic dataset before concatenation : ',x_quad.shape)
x_quad = df_21.join(x_quad) 
x_quad.dropna(inplace=True) 
print('Shape of quadratic dataset after concatenation : ',x_quad.shape) 

Shape of quadratic dataset before concatenation :  (22483, 104)
Shape of quadratic dataset after concatenation :  (22231, 106)


In [28]:
print('Shape of quadratic dataset before concatenation : ',xp_quad.shape)
xp_quad = df_1yr.join(xp_quad) 
xp_quad.dropna(inplace=True) 
print('Shape of quadratic dataset after concatenation : ',xp_quad.shape) 

Shape of quadratic dataset before concatenation :  (22483, 104)
Shape of quadratic dataset after concatenation :  (252, 106)


#### 5.3 Cubic dataset

In [29]:
xp_cubic = x_cubic.copy() 
print('Shape of cubic dataset before concatenation : ',x_cubic.shape)
x_cubic = df_21.join(x_cubic) 
x_cubic.dropna(inplace=True) 
print('Shape of cubic dataset after concatenation : ',x_cubic.shape) 

Shape of cubic dataset before concatenation :  (22483, 559)
Shape of cubic dataset after concatenation :  (22231, 561)


In [30]:
print('Shape of quadratic dataset before concatenation : ',xp_cubic.shape)
xp_cubic = df_1yr.join(xp_cubic) 
xp_cubic.dropna(inplace=True) 
print('Shape of quadratic dataset after concatenation : ',xp_cubic.shape) 

Shape of quadratic dataset before concatenation :  (22483, 559)
Shape of quadratic dataset after concatenation :  (252, 561)


### 7. Class for Training and Evaluating the Model

Model metrics calculated are cost function, accuracy, confusion matrix and classification report. 

To calculate the cost function, also known as the loss function, for logistic regression, we need to use the logistic loss function, which is commonly referred to as cross-entropy loss or log loss.

In [31]:
class logistic_regression:
    def __init__(self):
        self.train_size = 0.4
        self.random_state = 42 

    def scaling_x(self,X):
        scaler = StandardScaler()
        scaled_X = scaler.fit_transform(X)
        return scaled_X
    
    def cost_func(self,model,x_test,y_test): 
        probabilities = model.predict_proba(x_test)[:,1] # Getting probabilities for class 1 (positive class)
        cost = log_loss(y_test,probabilities) 
        return cost 

    def model_metrics(self,model,x_test,y_test):
        y_pred = model.predict(x_test) 
        cost_fn = self.cost_func(model,x_test,y_test)
        accuracy = accuracy_score(y_test,y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        class_report = classification_report(y_test, y_pred)

        print(f'Cost function : {cost_fn:.2f}') 
        print(f'Accuracy : {accuracy:.2f}')
        print('Confusion Matrix : ')
        print(conf_matrix) 
        print('Classification Report : ')
        print(class_report) 
    
    def training_model(self,df,df_exp):

        X = df.drop(columns=['Signal'],axis=1)
        Y = df['Signal']  

        scaled_X = self.scaling_x(X)
        # Split the data into initial training set (40%) and test set (60%)
        x_train, x_test, y_train, y_test = train_test_split(scaled_X,Y,train_size=0.4, shuffle=False, 
                                                            random_state=42)
        model = LogisticRegression(C=1.0)   # C is the regularization parameter
        
        model.fit(x_train,y_train) 

        print('='*20,'Metrics for Train set','='*20)
        print()    
        self.model_metrics(model,x_train,y_train) 
        print('='*20,'Metrics for Test set','='*20)
        print()    
        self.model_metrics(model,x_test,y_test) 
        print() 

        df_exp = df_exp.drop(columns=['Signal'],axis=1)
        y_pred = model.predict(df_exp) 
        return y_pred 
        
logistic = logistic_regression()    

### 8. Evaluation of Linear, Quadratic and Cubic Combination of features

#### 8.1 Evaluation on Linear Combination of features

In [32]:
yl_pred = logistic.training_model(x_linear,xp_linear)  

==================== Metrics for Train set ====================

Cost function : 0.03
Accuracy : 1.00
Confusion Matrix : 
[[4246    0]
 [  26 4620]]
Classification Report : 
              precision    recall  f1-score   support

          -1       0.99      1.00      1.00      4246
           1       1.00      0.99      1.00      4646

    accuracy                           1.00      8892
   macro avg       1.00      1.00      1.00      8892
weighted avg       1.00      1.00      1.00      8892

==================== Metrics for Test set ====================

Cost function : 0.06
Accuracy : 0.98
Confusion Matrix : 
[[4908    9]
 [ 226 8196]]
Classification Report : 
              precision    recall  f1-score   support

          -1       0.96      1.00      0.98      4917
           1       1.00      0.97      0.99      8422

    accuracy                           0.98     13339
   macro avg       0.98      0.99      0.98     13339
weighted avg       0.98      0.98      0.98     13339


In [33]:
xp_linear = xp_linear[['Momentum','Signal','Adj Close']] 
xp_linear['Predicted'] = yl_pred 
xp_linear 

,Momentum,Signal,Adj Close,Predicted
Date,,,,
2017-12-11,17.723153,1,2659.989990,-1
2017-12-12,18.039759,1,2664.110107,-1
2017-12-13,17.217357,1,2662.850098,-1
2017-12-14,17.695536,1,2652.010010,-1
2017-12-15,18.292420,1,2675.810059,-1
...,...,...,...,...
2018-12-04,2.296704,-1,2700.060059,-1
2018-12-06,2.524363,-1,2695.949951,-1
2018-12-07,0.144909,-1,2633.080078,-1


In [34]:
print('='*20,'Observed values for Signals','='*20) 
print(xp_linear['Signal'].value_counts()) 
print() 
print('='*20,'Predicted values for Signals','='*20) 
print(xp_linear['Predicted'].value_counts())  

==================== Observed values for Signals ====================
Signal
 1    233
-1     19
Name: count, dtype: int64

==================== Predicted values for Signals ====================
Predicted
-1    252
Name: count, dtype: int64


#### 8.2 Evaluation on Quadratic Combination of features

In [35]:
yq_pred = logistic.training_model(x_quad,xp_quad) 

==================== Metrics for Train set ====================

Cost function : 0.03
Accuracy : 1.00
Confusion Matrix : 
[[4233   13]
 [  29 4617]]
Classification Report : 
              precision    recall  f1-score   support

          -1       0.99      1.00      1.00      4246
           1       1.00      0.99      1.00      4646

    accuracy                           1.00      8892
   macro avg       1.00      1.00      1.00      8892
weighted avg       1.00      1.00      1.00      8892

==================== Metrics for Test set ====================

Cost function : 0.13
Accuracy : 0.97
Confusion Matrix : 
[[4669  248]
 [ 108 8314]]
Classification Report : 
              precision    recall  f1-score   support

          -1       0.98      0.95      0.96      4917
           1       0.97      0.99      0.98      8422

    accuracy                           0.97     13339
   macro avg       0.97      0.97      0.97     13339
weighted avg       0.97      0.97      0.97     13339


In [36]:
xp_quad = xp_quad[['Momentum','Signal','Adj Close']] 
xp_quad['Predicted'] = yq_pred 
xp_quad 

,Momentum,Signal,Adj Close,Predicted
Date,,,,
2017-12-11,17.723153,1,2659.989990,-1
2017-12-12,18.039759,1,2664.110107,-1
2017-12-13,17.217357,1,2662.850098,-1
2017-12-14,17.695536,1,2652.010010,-1
2017-12-15,18.292420,1,2675.810059,-1
...,...,...,...,...
2018-12-04,2.296704,-1,2700.060059,-1
2018-12-06,2.524363,-1,2695.949951,-1
2018-12-07,0.144909,-1,2633.080078,-1


In [37]:
print('='*20,'Observed values for Signals','='*20) 
print(xp_quad['Signal'].value_counts()) 
print() 
print('='*20,'Predicted values for Signals','='*20) 
print(xp_quad['Predicted'].value_counts())  

==================== Observed values for Signals ====================
Signal
 1    233
-1     19
Name: count, dtype: int64

==================== Predicted values for Signals ====================
Predicted
-1    252
Name: count, dtype: int64


#### 8.3 Evaluation on Cubic Combination of features

In [38]:
yc_pred = logistic.training_model(x_cubic,xp_cubic)  

==================== Metrics for Train set ====================

Cost function : 0.03
Accuracy : 1.00
Confusion Matrix : 
[[4236   10]
 [  28 4618]]
Classification Report : 
              precision    recall  f1-score   support

          -1       0.99      1.00      1.00      4246
           1       1.00      0.99      1.00      4646

    accuracy                           1.00      8892
   macro avg       1.00      1.00      1.00      8892
weighted avg       1.00      1.00      1.00      8892

==================== Metrics for Test set ====================

Cost function : 0.11
Accuracy : 0.97
Confusion Matrix : 
[[4629  288]
 [  81 8341]]
Classification Report : 
              precision    recall  f1-score   support

          -1       0.98      0.94      0.96      4917
           1       0.97      0.99      0.98      8422

    accuracy                           0.97     13339
   macro avg       0.97      0.97      0.97     13339
weighted avg       0.97      0.97      0.97     13339


In [39]:
xp_cubic = xp_cubic[['Momentum','Signal','Adj Close']] 
xp_cubic['Predicted'] = yc_pred 
xp_cubic 

,Momentum,Signal,Adj Close,Predicted
Date,,,,
2017-12-11,17.723153,1,2659.989990,-1
2017-12-12,18.039759,1,2664.110107,-1
2017-12-13,17.217357,1,2662.850098,-1
2017-12-14,17.695536,1,2652.010010,-1
2017-12-15,18.292420,1,2675.810059,-1
...,...,...,...,...
2018-12-04,2.296704,-1,2700.060059,-1
2018-12-06,2.524363,-1,2695.949951,-1
2018-12-07,0.144909,-1,2633.080078,-1


In [40]:
print('='*20,'Observed values for Signals','='*20) 
print(xp_cubic['Signal'].value_counts()) 
print() 
print('='*20,'Predicted values for Signals','='*20) 
print(xp_cubic['Predicted'].value_counts())  

==================== Observed values for Signals ====================
Signal
 1    233
-1     19
Name: count, dtype: int64

==================== Predicted values for Signals ====================
Predicted
-1    252
Name: count, dtype: int64


In [41]:
print('='*20,'Statistics of SPX for one year','='*20) 
print() 
calc_kpi = KPIs(databnh)
calc_kpi.calculate_kpi()  
print('='*20,'Statistics for Buy and Hold for one year revisiting signal every day','='*20) 
print() 
calc_kpi = KPIs(xp_cubic) 
calc_kpi.calculate_kpi()  

==================== Statistics of SPX for one year ====================

Annual Return : -1.0%
Sharpe Ratio : -0.03
Volatility : 16%
Maximum Drawdown : -10%
Average Daily Drawdown : -4%
==================== Statistics for Buy and Hold for one year revisiting signal every day ====================

Annual Return : -0.9%
Sharpe Ratio : -0.03
Volatility : 16%
Maximum Drawdown : -10%
Average Daily Drawdown : -4%


### 10. Sliding Window over Cubic Polynomials 

1. Training on an initial set of data (40% of data)
2. Training calibrate parameters before applying to testing sets
3. Convergence is monitored by cost function.
4. Convergence is achieved when (cost function < threshold) => Threshold=0.01
5. Convergence is checked every 50 days 
6. Window is slid forward after convergence is achieved 
7. Retraining is done every 8 years
8. Continue this till the end of data

This approach ensures that your model remains updated with recent data and can adapt to changing market conditions effectively

In [42]:
data.head() 

,Adj Close,momntm_30,momntm_60,momntm_90,momntm_120,momntm_180,momntm_270,momntm_300,momntm_360,drwdwn_15,drwdwn_60,drwdwn_90,drwdwn_120
Date,,,,,,,,,,,,,
1929-06-11,25.430000,-0.100000,-0.650000,-0.020000,2.990000,4.250000,5.070000,6.48,7.670000,-0.035647,-0.035647,-0.035647,-0.035647
1929-06-12,25.450001,-0.490000,-0.590000,-0.289999,2.750000,4.230001,5.010000,6.17,7.730001,-0.034888,-0.034888,-0.034888,-0.034888
1929-06-13,25.840000,-0.150000,-0.190001,0.000000,2.860001,4.480000,5.450001,6.93,8.290001,-0.020099,-0.020099,-0.020099,-0.020099
1929-06-14,25.930000,-0.180000,-0.090000,0.290001,2.860001,4.500000,5.880001,6.91,8.270000,-0.016686,-0.016686,-0.016686,-0.016686
1929-06-17,26.410000,0.039999,0.510000,0.709999,3.080000,5.000000,6.289999,7.42,8.910000,0.000000,0.000000,0.000000,0.000000


In [43]:
data['Momentum'] = (data['Adj Close'].pct_change(periods=252)*100).round(2)  # 252 days 
data.dropna(inplace=True)
data = generate_signals(data, 1, 5) 

In [44]:
data = data[data['Signal']!=0] 
data.head()

,Adj Close,momntm_30,momntm_60,momntm_90,momntm_120,momntm_180,momntm_270,momntm_300,momntm_360,drwdwn_15,drwdwn_60,drwdwn_90,drwdwn_120,Momentum,Signal
Date,,,,,,,,,,,,,,,
1930-06-16,20.559999,-3.010000,-3.580000,-2.750000,0.299999,-10.240000,-5.410000,-4.740000,-3.690001,-0.354677,-0.354677,-0.354677,-0.354677,-19.21,-1
1930-06-17,20.580000,-2.400000,-3.710001,-2.480000,0.190001,-10.270000,-5.510000,-4.469999,-3.590000,-0.354049,-0.354049,-0.354049,-0.354049,-20.36,-1
1930-06-18,19.860001,-3.900000,-4.420000,-3.109999,-0.980000,-11.269999,-5.529999,-4.969999,-4.680000,-0.376648,-0.376648,-0.376648,-0.376648,-23.41,-1
1930-06-19,20.790001,-2.519999,-3.459999,-2.330000,-0.029999,-9.480000,-4.910000,-3.839998,-3.789999,-0.347458,-0.347458,-0.347458,-0.347458,-21.28,-1
1930-06-20,20.250000,-3.150000,-4.370001,-3.070000,-0.639999,-9.910000,-4.510000,-4.530001,-4.290001,-0.364407,-0.364407,-0.364407,-0.364407,-23.35,-1


In [45]:
data['Signal'].value_counts() 

Signal
 1    13203
-1     9027
Name: count, dtype: int64

In [54]:
class Strategy:
    def __init__(self):
        self.conv_interval = 50  # days
        self.retrain_freq = 252*8   # 8 years = 8*252 days
        self.tolerance = 0.0001

    def scaling_x(self, X):
        scaler = StandardScaler()
        scaled_x = scaler.fit_transform(X)
        return scaled_x 

    def cost_funcn(self, model, X, Y):
        y_prob = model.predict_proba(X)[:,1]     # Getting probabilities for class 1 (positive class)
        cost = log_loss(Y, y_prob) 
        return cost     
    
    def data_concat(self, y_pred, start, end, df):

        datac = df[start:end] 
        y_pred = pd.Series(y_pred,index=datac.index) 

        bnch_df = datac[datac['Signal'] == 1].copy() 
        print('='*20,'Metric for bnch_df','='*20) 
        calc_kpi = KPIs(bnch_df)  
        calc_kpi.calculate_kpi() 

        log_indices = y_pred[y_pred==1].index 
        log_df = df.loc[log_indices].copy()  # Filter original DataFrame based on indices
        log_df['y_pred'] = y_pred[log_indices]     # Add y_pred column 
        log_df = log_df[['y_pred', 'Signal', 'Adj Close']] 
        # print(log_df.head(3))    
        print('='*20,'Metric for log_df','='*20) 
        calc_kpi = KPIs(log_df)  
        calc_kpi.calculate_kpi()
        
    def model_metrics(self, model, X, Y, start, end, df):
        y_pred = model.predict(X) 
        cost_fn = self.cost_funcn(model, X, Y)
        accuracy = accuracy_score(Y, y_pred)
        conf_matrix = confusion_matrix(Y, y_pred)
        class_report = classification_report(Y, y_pred)  
        
        print(f'Cost Function : {cost_fn}')
        print(f'Accuracy Score : {accuracy}')
        print('Confusion Matrix :') 
        print(conf_matrix) 
        print('Classification Report : ')
        print(class_report) 
        
        self.data_concat(y_pred, start, end, df)
    
    def date_correction(self,indx,df,num):
        idx1 = df.index.get_loc(indx)
        idx2 = idx1 + num 
        if idx2<len(df)-1:
            return idx2 
        else:
            return len(df)-1   
    
    def training_logistic(self, df):
        models = []     
        # Initialize Parameters        
        train_start = df.index[0]
        train_end = df.index[int(0.4*len(df))] 
        test_start = train_end 
        idx = df.index.get_loc(test_start)+252*8 
        test_end = df.index[idx] 

        model = LogisticRegression() 

        # Initial Training set
        x_train = df.loc[train_start:train_end].drop('Signal', axis=1) 
        xs_train = self.scaling_x(x_train) 
        y_train = df.loc[train_start:train_end, 'Signal'] 

        x_test = df.loc[test_start:test_end].drop('Signal', axis=1) 
        xs_test = self.scaling_x(x_test) 
        y_test = df.loc[test_start:test_end, 'Signal'] 

        model.fit(xs_train, y_train)

        print(f'Train set interval: {str(train_start).split(' 00:00:00')[0]} to {str(train_end).split(' 00:00:00')[0]}')       
        print() 
        print('='*20,'Metrics','='*20) 
        self.model_metrics(model, xs_train, y_train, train_start, train_end, df)

        print(f'Test set interval: {str(test_start).split(' 00:00:00')[0]} to {str(test_end).split(' 00:00:00')[0]}')       
        print()
        print('='*20,'Metrics','='*20) 
        self.model_metrics(model, xs_test, y_test, test_start, test_end, df)
         

        # Training Loop  
        while test_end<df.index[-1]:

            model.fit(xs_train, y_train)

            # Loop for checking convergence
            previous_cost = None 

            while test_end<df.index[-1]:
                x_test = df.loc[test_start:test_end].drop('Signal', axis=1) 
                xs_test = self.scaling_x(x_test) 
                y_test = df.loc[test_start:test_end, 'Signal'] 
                            
                current_cost = self.cost_funcn(model,xs_test,y_test) 

                if previous_cost is not None and (previous_cost-current_cost)/previous_cost < self.tolerance:
                    print() 
                    print(f'Convergence achieved at {str(test_end).split(' 00:00:00')[0]}') 
                    break   
                
                previous_cost = current_cost
                idx = self.date_correction(test_end,df,self.conv_interval)  
                test_end = df.index[idx] 

            # Slide the training window 
            idxt1 = self.date_correction(train_start, df,self.retrain_freq)
            train_start = df.index[idxt1] 

            idxt2 = self.date_correction(train_end, df,self.retrain_freq)
            train_end = df.index[idxt2]

            test_start = train_end 

            idxs = self.date_correction(test_end, df,self.retrain_freq)
            test_end = df.index[idxs]  

            # Updating training data
            
            if train_end<=df.index[-1]:
                x_train = df.loc[train_start:train_end].drop('Signal', axis=1) 
                xs_train = self.scaling_x(x_train) 
                y_train = df.loc[train_start:train_end, 'Signal'] 

            print(f'Train set interval: {str(train_start).split(' 00:00:00')[0]} to {str(train_end).split(' 00:00:00')[0]}')       
            print()
            print('='*20,'Metrics','='*20) 
            model_m = model.fit(xs_train,y_train) 
            models.append(model_m) 
            self.model_metrics(model_m, xs_train, y_train, train_start, train_end, df)   

            # Updating Testing data
            
            if test_end not in list(df.index):
                print(type(test_end))
                print(test_end)
                print(type(df.index[-1]))
                print(df.index[-1]) 
                print(df.index)  
                test_end = df.index[-1] 
                x_test = df.loc[test_start:test_end].drop('Signal', axis=1) 
                xs_test = self.scaling_x(x_test) 
                y_test = df.loc[test_start:test_end, 'Signal'] 
            else: 
                print(type(test_end))
                print(type(df.index[-1])) 
                x_test = df.loc[test_start:test_end].drop('Signal', axis=1) 
                xs_test = self.scaling_x(x_test) 
                y_test = df.loc[test_start:test_end, 'Signal'] 

            print(f'Test set interval: {str(test_start).split(' 00:00:00')[0]} to {str(test_end).split(' 00:00:00')[0]}')       
            print()
            print('='*20,'Metrics','='*20) 
            self.model_metrics(model_m, xs_test, y_test, test_start, test_end, df)   
            print('*'*100)
        return models 
        
strategy = Strategy()   

In [55]:
models = strategy.training_logistic(x_cubic)  

Train set interval: 1929-06-11 to 1964-12-15

==================== Metrics ====================
Cost Function : 0.029848665106413177
Accuracy Score : 0.991453952546947
Confusion Matrix :
[[4220   26]
 [  50 4597]]
Classification Report : 
              precision    recall  f1-score   support

          -1       0.99      0.99      0.99      4246
           1       0.99      0.99      0.99      4647

    accuracy                           0.99      8893
   macro avg       0.99      0.99      0.99      8893
weighted avg       0.99      0.99      0.99      8893

==================== Metric for bnch_df ====================
Annual Return : 6.6%
Sharpe Ratio : 0.25
Volatility : 26%
Maximum Drawdown : -78%
Average Daily Drawdown : -31%
==================== Metric for log_df ====================
Annual Return : 6.7%
Sharpe Ratio : 0.25
Volatility : 25%
Maximum Drawdown : -78%
Average Daily Drawdown : -31%
Test set interval: 1964-12-15 to 1973-01-23

==================== Metrics ===============

In [56]:
models 

[LogisticRegression(),
 LogisticRegression(),
 LogisticRegression(),
 LogisticRegression(),
 LogisticRegression(),
 LogisticRegression()]

### 10. Calculating Key Performance Indicators of various Logistic regression models

10.1 Benchmark SPX

10.2 Logistic Regression Linear Model

10.3 Logistic Regression Quadratic Model

10.4 Logistic Regression Cubic Model

### 11. Retraining the Model

#### Steps:

Frequency at which training set should be revised on regular intervals as new data is generated in market

1. Retraining period is about 5 and 10 years for one asset
2. But for a portfolio with multiple assets, this approach is not feasible